# GNINA

`gnina` performs **molecular docking** and predicts **binding affinity** between a protein and a small molecule. It leverages **deep learning models** to optimize docking poses and estimate **binding energy**. This function is based on: [GNINA: Deep Learning-Based Molecular Docking](https://pubs.acs.org/doi/10.1021/acs.jcim.0c01306).

### Inputs

* options: `dict`
* protein_trcs: `[TRC]`, the proteins to dock the smols onto.
* docking_sites: `[GninaDockingSite]`, 1-to-1 with the proteins, specifying the docking site location on the protein.
* smol_trcs: `[TRC]`, to dock all against each protein.
* ligand_libraries: `LigandLibrary` which provides a more storage-efficient way of passing very many smols for docking.

The docking_sites dictionary can either take the form of an "autobox":

| Name | Type | Default | Description |
|------|------|---------|-------------|
| autobox_ligand | `TR` | n/a | The reference ligand used to automatically determine the docking site box. |
| autobox_add | `Option<float>` | `none` | The additional padding distance to add around the autobox ligand. |

or the form of a `BindingSiteBoundingBox`.

### Outputs

* binding_site_results: `dict`
* 
The `binding_site_results` dictionary fields are as follows:

| Name | Type | Description |
|------|------|-------------|
| pose_trcs | `[TR]` | A list of TRs representing the docking poses. |
| scores | `dict` | A list of scoring results for each docking pose. |

and the `scores` dictionary fields are as follows:

| Name | Type | Description |
|------|------|-------------|
| mode | `int` | The pose mode number for this scoring result. |
| cnn_score | `float` | The CNN-based scoring value for the pose. |
| cnn_affinity | `float` | The CNN-predicted binding affinity for the pose. |
| affinity | `float` | The calculated binding affinity for the pose. |

### Options

| Name | Type | Default | Description |
|------|------|---------|-------------|
| minimize | `Option<bool>` | `none` | Whether to minimize the energy of the final pose. |
| exhaustiveness | `Option<uint>` | `none` | The search extent for the docking algorithm. |
| num_modes | `Option<uint>` | `none` | The number of dock poses to report. |
| covalent_rec_atom | `Option<str>` | `none` | The receptor atom that the ligand is bound to, specified as either "chain:resnum:atom_name" or x,y,z cartesian coordinates. |
| covalent_lig_atom_pattern | `Option<str>` | `none` | The SMARTS expression for the ligand atom that binds to the receptor. |
| covalent_lig_atom_position | `Option<str>` | `none` | The position of ligand in x,y,z cartesian coordinates. |
| covalent_optimize_lig | `Option<bool>` | `none` | Whether to optimize the covalent complex of ligand and residue with UFF. |

### Function usage

```haskell
let
    gnina = \protein_trc -> \bounding_box -> \smol_tr ->
        get 0 (get 0 (
            gnina_rex_s runspec {} [protein_trc] [bounding_box] [smol_tr] []
        ))
in
\protein_trc -> \bounding_box -> \smol_tr ->
    let
        result = gnina protein_trc bounding_box smol_tr
    in
        list_min (map (get "affinity") (get "scores" result)) {- get the best affinity value of all the docked poses -}
```